In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from datetime import datetime
import requests

In [12]:
# Buka halaman web
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options)
driver.get('https://radartasik.id/category/jawa-barat/kota-tasikmalaya/')

In [13]:
try:
    # Inisialisasi counter
    counter = 0
    # Loop sampai counter mencapai 10
    while counter < 10:
        # Temukan tombol "Muat Selanjutnya" atau "Lihat Selengkapnya"
        load_more_button = driver.find_element(By.CLASS_NAME, 'view-more-button')
        if load_more_button:
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)  # Scroll ke tombol
            load_more_button.click()  # Klik tombol
            time.sleep(30)  # Tunggu konten dimuat
            counter += 1  # Tambah counter setiap kali tombol diklik
        else:
            print("Tombol 'Muat Selanjutnya' tidak ditemukan atau tidak ada lebih banyak konten untuk dimuat.")
            break
except Exception as e:
    print("Terjadi kesalahan:", e)

In [14]:
# Ambil sumber halaman setelah JavaScript dijalankan
html = driver.page_source

# Parsing HTML dengan BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Cari elemen-elemen yang diinginkan, misalnya berita highlight
articles = soup.find_all('article')

In [26]:
def custom_strptime_pr(date_str):
    # Mengonversi bulan ke angka
    bulan_dict = {
        "Januari": "01",
        "Februari": "02",
        "Maret": "03",
        "April": "04",
        "Mei": "05",
        "Juni": "06",
        "Juli": "07",
        "Agustus": "08",
        "September": "09",
        "Oktober": "10",
        "November": "11",
        "Desember": "12"
    }
    for bulan, angka in bulan_dict.items():
        date_str = date_str.replace(bulan, angka)

    # Mengonversi ke format datetime
    date_time_obj = datetime.strptime(date_str, "%d %m %Y")
    return date_time_obj

In [31]:
# Buat list untuk menyimpan data
data = []
for article in articles:
    link = article.find('a')['href']
    judul = article.find('h2', attrs={'class': 'entry-title'}).string.strip()
    date_str = article.find('time', attrs={'class': 'entry-date'}).get_text(strip=True)
    # # Menggunakan fungsi custom_strptime_pr untuk mengkonversi tanggal
    date = custom_strptime_pr(date_str)
    print("Link:", link)
    print("Judul:", judul)
    print("Tanggal publish:", date_str)
    print(date)

    # get content
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    webpage = requests.get(link, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Mencari elemen konten berita yang sesuai
    article_content = soup.find('article', class_='post').find_all('p')

    # Mengambil teks dari elemen konten
    content_text = []
    for paragraph in article_content:
        content_text.append(paragraph.get_text(strip=True))

    # Menggabungkan semua teks konten
    article_text = ' '.join(content_text)
    print(article_text)
            
    # Menambahkan data ke list
    data.append({"url": link, "title": judul, "date": date, "content": article_content})

Link: https://radartasik.id/ct-scan-masih-rusak-banyak-pasien-rsud-dr-soekardjo/
Judul: CT Scan Masih Rusak, Banyak Pasien RSUD dr Soekardjo Tasikmalaya Terpaksa Harus Bolak-Balik
Tanggal publish: 8 November 2023
2023-11-08 00:00:00
TASIKMALAYA, RADARTASIK.ID– Layanan RSUD dr Soekardjo saat ini masih terhambat oleh kerusakan CT Scan. Di mana pasien harus menjalani pemeriksaan medis itu ke rumah sakit swasta. Persoalan pelayanan RSUD mengenai CT Scan tersebut cukup banyak dikeluhkan. Selain tidak praktis, biaya pemeriksaan keluar masuk rumah sakit pun terdapat biaya. Kondisi tersebut sudah terjadi bertahun-tahun dan membuat layanan RSUD dr Soekardjo menjadi tidak maksimal. Karena ketersediaan sarana dan prasarana dalam pelayanan medis menjadi sebuah hal yang vital. Baca juga :Parodikan Lagu Ikan Dalam Kolam, Guru Agama di Tasikmalaya Viral di Tiktok Terkait hal itu Wadir Umum RSUD dr Soekardjo Budi Martanova mengakui bahwa CT Scan memang kondisinya rusak. Sehingga mau tidak mau pasien h

ConnectionError: HTTPSConnectionPool(host='radartasik.id', port=443): Max retries exceeded with url: /pohon-di-depan-bank-mandiri-terbakar-diduga-akibat/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9a80e5ab50>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [34]:
import pandas as pd
# Membuat DataFrame dari list data
df = pd.DataFrame(data)
df

,url,title,date,content
0,https://radartasik.id/ct-scan-masih-rusak-bany...,"CT Scan Masih Rusak, Banyak Pasien RSUD dr Soe...",2023-11-08,"[[[TASIKMALAYA, RADARTASIK.ID], – Layanan RSU..."
1,https://radartasik.id/harga-cabai-rawit-bisa-t...,Gak Bahaya Ta? Harga Cabai Rawit Bisa Terus-Te...,2023-11-08,"[[[TASIKMALAYA, RADARTASIK.ID], – Masalah ken..."
2,https://radartasik.id/raperda-penataan-kelemba...,Raperda Penataan Kelembagaan Kota Tasikmalaya ...,2023-11-08,"[[[JAKARTA, RADARTASIK.ID], – Raperda penataa..."
3,https://radartasik.id/pakar-ekonomi-ingatkan-p...,Pakar Ekonomi Ingatkan Pemkot Tasikmalaya Soal...,2023-11-08,"[[[TASIKMALAYA, RADARTASIK.ID], – Pakar ekono..."
4,https://radartasik.id/bpbd-kota-tasikmalaya-mi...,BPBD Kota Tasikmalaya Minta Warga Waspadai Huj...,2023-11-08,"[[[TASIKMALAYA, RADARTASIK.ID], – BPBD Kota T..."
...,...,...,...,...
76,https://radartasik.id/sekda-buka-suara-soal-te...,Sekda Buka Suara Soal Temuan BPK dan Perkara H...,2023-11-01,"[[[TASIKMALAYA, RADARTASIK.ID], – Sekretaris ..."
77,https://radartasik.id/ketua-kpu-kota-tasikmala...,Ketua KPU Kota Tasikmalaya yang Baru Bakal Pan...,2023-11-01,"[[[TASIKMALAYA, RADARTASIK.ID], –Setelah terj..."
78,https://radartasik.id/program-penanggulangan-k...,Program Penanggulangan Kemiskinan di Instansi ...,2023-11-01,"[[[TASIKMALAYA, RADARTASIK.ID], – Pemerhati K..."
79,https://radartasik.id/melobi-pdam-tirta-sukapu...,Melobi PDAM Tirta Sukapura Lebih Rasional Diba...,2023-11-01,"[[[TASIKMALAYA, RADARTASIK.ID], – Keinginan m..."
